In [1]:
import league
import players
import statistics
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from plotly.subplots import make_subplots
from prettytable import PrettyTable

PLAYERS = players.Players()
LEAGUE = league.League(12, "649912836461539328", PLAYERS)

# Playoff Probability (part 2)

After last weeks hit notebook on playoff probabilities, we are back again to predict the playoff picture now that we have one additional week of information. The standings at present are as follows (and yes, Stephane is in a playoff position):

```
Noe's Loyal Lighting Salesman Division
1. Nahome (18-6)
2. Ben (10-14)
3. Stephane (9-15)
4. Noe (9-15)
5. Will (7-17)

Noah's SideThiccs
1. Praveen (16-8)
2. Stian (15-9)
3. Tom (13-11)
4. Noah (13-11)
5. Tegran (10-14)
```

Power Rankings for this week are as follows

Power rankings for this week are as follows

```
1. Nahome
2. Stian
3. Stephane
4. Praveen
5. Tom
6. Tegran
7. Noe
8. Ben
9. Noah
10. Will
```

Stephane is up to his highest ranking of the year while Nahome and Stian are both holding steady at the top. Praveen drops two spots after an abysmal week, while Noah is only just ahead of Will as he looks to stop a six game skid this weekend.

## Methodology

I think the analysis was pretty good last week, so see that notebook for notes on my methodology. This week I'll simply compare the new projections to last weeks and see who is improving their odds. An example simulation is shown below.

In [2]:
LEAGUE.sim_remaining_season(np.random.normal)
results = LEAGUE.simmed_results()

for division in results:
    division = sorted(division, reverse=True)
    for t in division:
        print(LEAGUE.name_pairings[LEAGUE.roster_pairings[t[4]]], t[0], t[1], t[2])
    print()

AssNTitties 22 6 1847.2985689819016
My Goat Loves You 13 15 1761.6879292051883
the noés 11 17 1568.8594576280016
billcap 10 18 1563.876102931355
RB sanctuary 9 19 1581.379947937891

Tree Leaves 20 8 1775.3664840151307
SideThicc #2 16 12 1749.75872353108
tealeaves 16 12 1735.2563000282448
Thicc King 13 15 1652.334908460366
The Brutherhood 10 18 1649.1772991488147



To simulate 50,000 seasons again!

In [3]:
# takes awhile to run ~ 15 min
simulation = []
for i in range(50000):
    LEAGUE.sim_remaining_season(np.random.normal)
    simulation.append(LEAGUE.simmed_results())

Now lets see where every team finished in this new simulation.

In [4]:
final_standing = {t.roster_id: [0,0,0,0,0] for t in LEAGUE.standings}
probabilities = {t.roster_id: 0 for t in LEAGUE.standings}

for result in simulation:
    for division in result:
        division = sorted(division, reverse=True)
        for i, t in enumerate(division):
            final_standing[t[4]][i] += 1

for roster_id in final_standing:
    print(LEAGUE.name_pairings[LEAGUE.roster_pairings[roster_id]], final_standing[roster_id])
    probabilities[roster_id] = sum(
        final_standing[roster_id][0:3]) / sum(final_standing[roster_id]) * 100
    print(sum(final_standing[roster_id][0:3]) / sum(final_standing[roster_id]))

finish_probabilities = {roster_id: [(num / sum(final_standing[roster_id])) * 100 for num in final_standing[roster_id]] for roster_id in final_standing}

for id in probabilities:
    probabilities[id] = round(probabilities[id], 2)

for id in finish_probabilities:
    finish_probabilities[id] = [round(prob, 2) for prob in finish_probabilities[id]]

AssNTitties [50000, 0, 0, 0, 0]
1.0
Tree Leaves [31665, 14846, 3305, 184, 0]
0.99632
tealeaves [17326, 25618, 6053, 1003, 0]
0.97994
SideThicc #2 [708, 5462, 22240, 18775, 2815]
0.5682
Thicc King [301, 4074, 18106, 24310, 3209]
0.44962
My Goat Loves You [0, 36200, 10447, 3296, 57]
0.93294
The Brutherhood [0, 0, 296, 5728, 43976]
0.00592
RB sanctuary [0, 8721, 22593, 15988, 2698]
0.62628
the noés [0, 4646, 13561, 25920, 5873]
0.36414
billcap [0, 433, 3399, 4796, 41372]
0.07664


Now lets visualize these finish positions as bars of outcomes

In [5]:
fig = make_subplots(rows=5, cols=2, shared_yaxes=True,
                    subplot_titles=LEAGUE.get_names_for_plotting())

teams = [team for team in LEAGUE.standings]

for i in range(5):
    for j in range(1, 3):
        current = teams[(i+i+j)-1]
        fig.add_trace(
            go.Bar(y=final_standing[current.roster_id],
                   x=["1st", "2nd", "3rd", "4th", "5th"]),
            i+1, j
        )

fig.update_layout(title="Likelihood of each Teams' Final Standing",
                  showlegend=False,
                  height=2000, width=1500
                  )

fig.show()

Below I print out the calculated probability that each team makes the playoffs and the probability they finish in each of the five division positions. I also include the change in playoff odds from last week. 

In [10]:
t = PrettyTable(['Team Name', "Percent Chance of Making Playoffs", "Percent Change from Last Week"])
last_week = {1: 99.93, 2: 3.77, 3: 87.13, 4: 99.93, 5: 47.47,
             6: 26.38, 7: 100, 8: 82.79, 9: 28.61, 10: 25.4}
for team in LEAGUE.standings:
    t.add_row([team.name, probabilities[team.roster_id], round(probabilities[team.roster_id] - last_week[team.roster_id], 2)])
    
print(t)

+-------------------+-----------------------------------+-------------------------------+
|     Team Name     | Percent Chance of Making Playoffs | Percent Change from Last Week |
+-------------------+-----------------------------------+-------------------------------+
|    AssNTitties    |               100.0               |              0.0              |
|    Tree Leaves    |               99.63               |              -0.3             |
|     tealeaves     |               97.99               |             10.86             |
|    SideThicc #2   |               56.82               |             30.44             |
|     Thicc King    |               44.96               |             -37.83            |
| My Goat Loves You |               93.29               |             -6.64             |
|  The Brutherhood  |                0.59               |             -3.18             |
|    RB sanctuary   |               62.63               |             37.23             |
|      the

In [11]:
t = PrettyTable(['Team Name', "P(Finishing 1st in Division)",
                "P(Finishing 2nd in Division)", "P(Finishing 3rd in Division)", "P(Finishing 4th in Division)", "P(Finishing 5th in Division)"])
for team in LEAGUE.standings:
    t.add_row([team.name, finish_probabilities[team.roster_id]
              [0], finish_probabilities[team.roster_id][1], finish_probabilities[team.roster_id][2], finish_probabilities[team.roster_id][3], finish_probabilities[team.roster_id][4]])

print(t)


+-------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+
|     Team Name     | P(Finishing 1st in Division) | P(Finishing 2nd in Division) | P(Finishing 3rd in Division) | P(Finishing 4th in Division) | P(Finishing 5th in Division) |
+-------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+
|    AssNTitties    |            100.0             |             0.0              |             0.0              |             0.0              |             0.0              |
|    Tree Leaves    |            63.33             |            29.69             |             6.61             |             0.37             |             0.0              |
|     tealeaves     |            34.65             |            51.24             |            12.11             | 